# Get Sales interaction Data

In [43]:
import pandas as pd
import urllib
from sqlalchemy import create_engine

connection_string = (
    "DRIVER={SQL Server};"
    "SERVER=web.speed.live;"
    "DATABASE=Sanad1;"
    "Trusted_Connection=yes;"
)

params = urllib.parse.quote_plus(connection_string)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# Sales Data (Positive Interactions)
query = """
SELECT 
    c.Customer_B2B_ID,
    s.ItemId,
    c.GOVERNER_NAME,
    c.SALES_CHANNEL_CODE,
    i.DESCRIPTION,
    CASE 
        WHEN CHARINDEX('|', i.MASTER_BRAND) > 0 
        THEN RIGHT(i.MASTER_BRAND, LEN(i.MASTER_BRAND) - CHARINDEX('|', i.MASTER_BRAND))
        ELSE i.MASTER_BRAND
    END AS Brand,
    CASE 
        WHEN CHARINDEX('|', i.MG2) > 0 
        THEN RIGHT(i.MG2, LEN(i.MG2) - CHARINDEX('|', i.MG2))
        ELSE i.MG2
    END AS Category,
    1 AS Label
FROM MP_Sales s
JOIN MP_Customers c ON s.CustomerId = c.SITE_NUMBER
JOIN MP_Items i ON s.ItemId = i.ITEM_CODE
WHERE s.NETSALESVALUE > 0 
  AND YEAR(Date) = YEAR(GETDATE()) 
  AND MONTH(Date) = 8
"""

# Customers & Items Metadata
Customers_query = "SELECT Customer_B2B_ID, GOVERNER_NAME, SALES_CHANNEL_CODE FROM MP_Customers"
items_query = """
SELECT 
    ITEM_CODE as ItemId,
    DESCRIPTION,
    CASE 
        WHEN CHARINDEX('|', MASTER_BRAND) > 0 
        THEN RIGHT(MASTER_BRAND, LEN(MASTER_BRAND) - CHARINDEX('|', MASTER_BRAND))
        ELSE MASTER_BRAND
    END AS Brand,
    CASE 
        WHEN CHARINDEX('|', MG2) > 0 
        THEN RIGHT(MG2, LEN(MG2) - CHARINDEX('|', MG2))
        ELSE MG2
    END AS Category
FROM MP_Items
"""

# Read from database
df = pd.read_sql(query, engine)
df_customers = pd.read_sql(Customers_query, engine)
df_items = pd.read_sql(items_query, engine)


In [45]:
import random

# Ensure ItemId is string for consistency
df["ItemId"] = df["ItemId"].astype(str)
df_items["ItemId"] = df_items["ItemId"].astype(str)

# List of all customers and items
all_customers = df['Customer_B2B_ID'].unique()
all_items = df_items['ItemId'].unique()
positive_set = set(zip(df['Customer_B2B_ID'], df['ItemId']))

negative_samples = []

# Sample 3x negatives for every customer based on items they didn’t buy
for customer in all_customers:
    bought_items = df[df['Customer_B2B_ID'] == customer]['ItemId'].unique()
    not_bought_items = list(set(all_items) - set(bought_items))
    
    if not not_bought_items:
        continue

    for _ in range(3 * len(bought_items)):
        negative_item = random.choice(not_bought_items)
        negative_samples.append((customer, negative_item))

# Create negative DataFrame
df_negative = pd.DataFrame(negative_samples, columns=['Customer_B2B_ID', 'ItemId'])
df_negative['Label'] = 0

# Join metadata
df_negative = df_negative.merge(df_customers, on='Customer_B2B_ID', how='left')
df_negative = df_negative.merge(df_items, on='ItemId', how='left')


In [46]:
df_all = pd.concat([df, df_negative], ignore_index=True)


In [47]:
print("Total interactions:", len(df_all))
print("Positive samples:", df_all['Label'].sum())
print("Negative samples:", (df_all['Label'] == 0).sum())


Total interactions: 421671
Positive samples: 80706
Negative samples: 340965


In [48]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
categorical_cols = ['Customer_B2B_ID', 'GOVERNER_NAME', 'SALES_CHANNEL_CODE',
                    'ItemId', 'Brand', 'Category', 'DESCRIPTION']

# Convert ItemId to string (ensure consistent types)
df_all["ItemId"] = df_all["ItemId"].astype(str)

# Apply LabelEncoder to all categorical columns
for col in categorical_cols:
    le = LabelEncoder()
    df_all[col] = le.fit_transform(df_all[col].astype(str))
    label_encoders[col] = le  # Save encoder for later use


In [49]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    df_all, 
    test_size=0.2, 
    random_state=42, 
    stratify=df_all['Label']
)


In [50]:
print("Train size:", len(train_df))
print("Validation size:", len(val_df))


Train size: 337336
Validation size: 84335


In [51]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dot
from tensorflow.keras.models import Model

# Define embedding dimensions
embedding_dim = 32
num_users = df_all['Customer_B2B_ID'].nunique()
num_govern = df_all['GOVERNER_NAME'].nunique()
num_channels = df_all['SALES_CHANNEL_CODE'].nunique()
num_items = df_all['ItemId'].nunique()
num_brands = df_all['Brand'].nunique()
num_categories = df_all['Category'].nunique()

# User Tower
user_id_in = Input(shape=(1,), name="user_id_in")
govern_in = Input(shape=(1,), name="govern_in")
channel_in = Input(shape=(1,), name="channel_in")

user_emb = Embedding(input_dim=num_users, output_dim=embedding_dim)(user_id_in)
gov_emb = Embedding(input_dim=num_govern, output_dim=8)(govern_in)
channel_emb = Embedding(input_dim=num_channels, output_dim=8)(channel_in)

user_vec = Concatenate()([
    Flatten()(user_emb),
    Flatten()(gov_emb),
    Flatten()(channel_emb)
])
user_vec = Dense(64, activation='relu')(user_vec)

# Item Tower
item_id_in = Input(shape=(1,), name="item_id_in")
brand_in = Input(shape=(1,), name="brand_in")
category_in = Input(shape=(1,), name="category_in")

item_emb = Embedding(input_dim=num_items, output_dim=embedding_dim)(item_id_in)
brand_emb = Embedding(input_dim=num_brands, output_dim=8)(brand_in)
cat_emb = Embedding(input_dim=num_categories, output_dim=8)(category_in)

item_vec = Concatenate()([
    Flatten()(item_emb),
    Flatten()(brand_emb),
    Flatten()(cat_emb)
])
item_vec = Dense(64, activation='relu')(item_vec)

# Combine towers
dot_product = Dot(axes=1)([user_vec, item_vec])
output = Dense(1, activation='sigmoid')(dot_product)

# Build model
model = Model(
    inputs=[user_id_in, govern_in, channel_in, item_id_in, brand_in, category_in],
    outputs=output
)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_id_in (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 govern_in (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 channel_in (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 item_id_in (InputLayer)        [(None, 1)]          0           []                               
                                                                                            

In [52]:
history = model.fit(
    x=[
        train_df['Customer_B2B_ID'],
        train_df['GOVERNER_NAME'],
        train_df['SALES_CHANNEL_CODE'],
        train_df['ItemId'],
        train_df['Brand'],
        train_df['Category']
    ],
    y=train_df['Label'],
    validation_data=(
        [
            val_df['Customer_B2B_ID'],
            val_df['GOVERNER_NAME'],
            val_df['SALES_CHANNEL_CODE'],
            val_df['ItemId'],
            val_df['Brand'],
            val_df['Category']
        ],
        val_df['Label']
    ),
    epochs=20,
    batch_size=512
)


Epoch 1/20
659/659 [==============================] - 6s 7ms/step - loss: 0.2638 - auc: 0.9400 - val_loss: 0.2060 - val_auc: 0.9682
Epoch 2/20
659/659 [==============================] - 5s 7ms/step - loss: 0.1868 - auc: 0.9699 - val_loss: 0.1856 - val_auc: 0.9698
Epoch 3/20
659/659 [==============================] - 4s 7ms/step - loss: 0.1642 - auc: 0.9741 - val_loss: 0.1711 - val_auc: 0.9719
Epoch 4/20
659/659 [==============================] - 6s 9ms/step - loss: 0.1463 - auc: 0.9783 - val_loss: 0.1629 - val_auc: 0.9739
Epoch 5/20
659/659 [==============================] - 6s 9ms/step - loss: 0.1309 - auc: 0.9821 - val_loss: 0.1585 - val_auc: 0.9749
Epoch 6/20
659/659 [==============================] - 6s 10ms/step - loss: 0.1172 - auc: 0.9853 - val_loss: 0.1575 - val_auc: 0.9747
Epoch 7/20
659/659 [==============================] - 5s 7ms/step - loss: 0.1054 - auc: 0.9876 - val_loss: 0.1573 - val_auc: 0.9745
Epoch 8/20
659/659 [==============================] - 5s 8ms/step - loss: 0

In [ ]:
import numpy as np

# Utility function to safely transform only known values
def safe_transform(le, values):
    known = set(le.classes_)
    filtered = [v for v in values if v in known]
    return le.transform(filtered), filtered

def recommend_top_popular_items(top_n=5):
    top_items = (
        df.groupby(["ItemId", "DESCRIPTION"])
          .size()
          .reset_index(name="purchase_count")
          .sort_values("purchase_count", ascending=False)
          .head(top_n)
    )
    print("📦 Recommending top popular products instead.")
    top_items["score"] = "popular"
    return top_items[["ItemId", "DESCRIPTION", "score"]]


def recommend_top_items(customer_id, top_n=5):
    # Step 1: Get customer metadata
    customer_row = df_customers[df_customers["Customer_B2B_ID"] == customer_id]
    if customer_row.empty:
        print(f"❌ No metadata found for Customer ID: {customer_id}")
        return None
    customer_row = customer_row.iloc[0]

    

    # Step 2: Check if customer was in training
    if customer_id not in label_encoders["Customer_B2B_ID"].classes_:
        print(f"⚠️ Customer ID {customer_id} not in training — using popular fallback.")
        return recommend_top_popular_items(top_n)

    # Step 3: Encode customer features
    encoded_customer_id = label_encoders["Customer_B2B_ID"].transform([customer_id])[0]
    encoded_govern = label_encoders["GOVERNER_NAME"].transform([customer_row["GOVERNER_NAME"]])[0]
    encoded_channel = label_encoders["SALES_CHANNEL_CODE"].transform([customer_row["SALES_CHANNEL_CODE"]])[0]

    # Step 4: Clean item metadata from df_all
    df_all["ItemId"] = df_all["ItemId"].astype(str)
    item_meta = df_all[["ItemId", "DESCRIPTION", "Brand", "Category"]].drop_duplicates()
    item_meta["Brand"] = item_meta["Brand"].astype(str)
    item_meta["Category"] = item_meta["Category"].astype(str)

    # Step 5: Filter out already purchased items
    already_bought = df[df["Customer_B2B_ID"] == customer_id]["ItemId"].astype(str).unique()
    filtered_items = item_meta[~item_meta["ItemId"].isin(already_bought)].copy()

    if filtered_items.empty:
        print("⚠️ Customer has already purchased all known items — fallback to popular.")
        return recommend_top_popular_items(top_n)

    # Step 6: Safe encode Brand & Category
    try:
        encodable_items = filtered_items[
            filtered_items["Brand"].isin(label_encoders["Brand"].classes_) &
            filtered_items["Category"].isin(label_encoders["Category"].classes_)
        ].copy()

        if encodable_items.empty:
            print("❌ No encodable items for this customer — fallback to popular.")
            return recommend_top_popular_items(top_n)

        # Transform each column
        encodable_items["ItemId_enc"] = label_encoders["ItemId"].transform(encodable_items["ItemId"])
        encodable_items["Brand_enc"] = label_encoders["Brand"].transform(encodable_items["Brand"])
        encodable_items["Category_enc"] = label_encoders["Category"].transform(encodable_items["Category"])

    except Exception as e:
        print("❌ Encoding error:", e)
        return recommend_top_popular_items(top_n)

    # Step 7: Create model input
    num_items = len(encodable_items)
    user_input = [
        np.array([encoded_customer_id] * num_items),
        np.array([encoded_govern] * num_items),
        np.array([encoded_channel] * num_items),
        encodable_items["ItemId_enc"].values,
        encodable_items["Brand_enc"].values,
        encodable_items["Category_enc"].values
    ]

    # Step 8: Predict scores
    predictions = model.predict(user_input, batch_size=512)
    encodable_items["score"] = predictions

    # Step 9: Sort and return top N
    top_items = encodable_items.sort_values("score", ascending=False).head(top_n)
    return top_items[["ItemId", "DESCRIPTION", "score"]]


In [71]:
top_recs = recommend_top_items("16442", top_n=5)
print(top_recs)


❌ No encodable items for this customer — fallback to popular.
📦 Recommending top popular products instead.
             ItemId                           DESCRIPTION    score
1168  6223001366362         بيبسي - مشروب غازي - كانز جيب  popular
1838  6224008033486          فولت مشروب طاقه 200 مل *12 ق  popular
1220  6223001875208          بيتي - عصير - مانجو - 235 مل  popular
1167  6223001366355  بيبسي - مشروب غازي - عائلي - 1.5 لتر  popular
1171  6223001366621    ستينج - مشروب طاقه - زجاج - 275 مل  popular


In [72]:
df_all[df_all['Label'] == 1]

,Customer_B2B_ID,ItemId,GOVERNER_NAME,SALES_CHANNEL_CODE,DESCRIPTION,Brand,Category,Label
0,4220,8160,0,5,902,3,5,1
1,456,300,1,5,4431,56,4,1
2,456,348,1,5,4894,56,4,1
3,4603,4119,8,5,1154,84,2,1
4,4603,3713,8,5,1082,213,2,1
...,...,...,...,...,...,...,...,...
80701,615,4272,1,5,7859,230,6,1
80702,5758,4828,0,5,1989,93,4,1
80703,583,6719,1,5,4558,229,4,1
80704,583,7869,1,5,4557,229,4,1
